In [ ]:
import json
import pandas as pd
import numpy as np
from datetime import datetime
from utils import Normalizer
from sklearn.preprocessing import LabelEncoder
from transformers import BertModel, BertTokenizer
import torch
from tqdm import tqdm
from utils import read_jsonl, write_jsonl
import os

In [4]:
data = pd.read_excel('./数据/count-result-final-drop-space.xlsx')


In [5]:
data

,studentid,userid,courseid,Involvement,Concentration,Persistence,assignment_time,Interaction,Positive,Negative,Confusion,Active,Constructive,Interactive,total,Learning_achivement
0,0,7653,TsinghuaX/20220332_1X/_,17,7833.784972,29,24808.117151,52,29,5,18,1,27,24,99.000000,0
1,1,4069,TsinghuaX/20220332_1X/_,0,0.000000,19,4729.089555,8,2,0,3,4,0,1,92.500000,0
2,2,19624,TsinghuaX/20220332_1X/_,20,43412.307920,24,40966.697621,14,2,2,10,0,8,6,91.611111,0
3,3,7580,TsinghuaX/20220332_1X/_,12,8772.234518,25,28445.668263,8,6,1,1,0,6,2,90.507122,0
4,4,8104,TsinghuaX/20220332_1X/_,31,39269.612402,45,42163.801295,14,4,3,7,1,4,9,89.403846,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59576,85872,16915,TsinghuaX/80512073X/_,0,0.000000,0,0.000000,1,1,0,0,1,0,0,0.000000,1
59577,85875,63285,TsinghuaX/80512073X/_,0,0.000000,1,32.470682,0,0,0,0,0,0,0,0.000000,1
59578,85877,33606,TsinghuaX/80512073X/_,5,7451.025834,0,0.000000,0,0,0,0,0,0,0,0.000000,1
59579,85878,84322,TsinghuaX/80512073X/_,1,705.422000,1,35.591872,0,0,0,0,0,0,0,0.000000,1


# 数据读取与一些预处理

In [6]:
data = pd.read_excel('./数据/count-result-final-drop-space.xlsx')


user = pd.read_excel('./数据/auth_user.xlsx')
user['year_of_birth'] = user['year_of_birth'].fillna(user['year_of_birth'].median())
user['year_of_birth'][user['year_of_birth']<1950] = 1950
user['year_of_birth'] = user['year_of_birth'] - user['year_of_birth'].min()
user['gender'] = user['gender'].fillna('UNK')
user['level_of_education'] = user['level_of_education'].fillna('UNK')


forum = pd.read_excel('./数据/forum.xlsx')
forum['updated'] = forum['updated'].replace('None', None)
def parse_date(date_str):
    if date_str is None:
        return None
    return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%f%z")
# 将数据列表转换为datetime对象列表
datetime_list = [parse_date(item) for item in forum['updated'].values]
forum['updated'] = datetime_list
forum = forum.sort_values(by=['userid', 'courseid', 'updated'])
action = pd.read_excel('./数据/log.xlsx')
action = action.sort_values(by=['userid', 'date'])

C:\Users\zexuan\AppData\Local\Temp\ipykernel_7988\1942904836.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user['year_of_birth'][user['year_of_birth']<1950] = 1950
D:\anaconda3\envs\torch\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [11]:
forum

,id,userid,courseid,type,comment_thread_id,created,updated,last_activity,title,body,comment_count,vote_up,vote_down
66,52295d5dd4dd12277c000001,1,tsinghuax/cs001/test,CommentThread,NaN,2013-09-06T12:43:09.575+0800,2013-09-06 12:43:09.575000+08:00,2013-09-06T13:50:36.015+0800,test,!!,4.0,0.0,0.0
67,52295d701daa980f45000001,1,tsinghuax/cs001/test,Comment,52295d5dd4dd12277c000001,2013-09-06T12:43:28.876+0800,2013-09-06 12:43:28.876000+08:00,NaN,NaN,123,0.0,0.0,0.0
68,52296c15d4dd12277c000005,2,tsinghuax/cs001/test,Comment,52295d5dd4dd12277c000001,2013-09-06T13:45:57.768+0800,2013-09-06 13:45:57.768000+08:00,NaN,NaN,test,0.0,0.0,0.0
70,52296d1cd4dd12277c000007,2,tsinghuax/cs001/test,Comment,52295d5dd4dd12277c000001,2013-09-06T13:50:20.670+0800,2013-09-06 13:50:20.670000+08:00,NaN,NaN,test,0.0,0.0,0.0
69,52296c45d4dd12277c000006,2,tsinghuax/cs001/test,Comment,52295d5dd4dd12277c000001,2013-09-06T13:46:45.439+0800,2013-09-06 13:50:35.980000+08:00,NaN,NaN,hello,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70638,542c199df605abc09b00688e,325678,TsinghuaX/30700313_2014X/2014_T2,Comment,542c0ebbf605abc015006841,2014-10-01T23:11:25.906+0800,2014-10-01 23:11:25.906000+08:00,NaN,NaN,1.查了下，有一门科学叫神经心理学...应该是它的范畴； 2.和知觉有关吧，我自己的解析就是...,0.0,0.0,0.0
70542,542c087cf605ab597700681c,325709,TsinghuaX/30700313_2014X/2014_T2,Comment,542bf543459f08670700674a,2014-10-01T21:58:20.030+0800,2014-10-01 21:58:20.030000+08:00,NaN,NaN,幸福和快乐的最大区别在于：快乐总是伴随着事情的发生而发生，而幸福往往是在事情结束后对快乐的经...,0.0,0.0,0.0
70627,542c1723f605ab312e006878,325842,TsinghuaX/30700313_2014X/2014_T2,Comment,542c0ebbf605abc015006841,2014-10-01T23:00:51.818+0800,2014-10-01 23:00:51.818000+08:00,NaN,NaN,要完成作业得查资料吧。,0.0,0.0,0.0
70705,542c2582f605ab48000068bf,325944,TsinghuaX/30700313_2014X/2014_T2,Comment,542a78caf605ab40ab006432,2014-10-02T00:02:10.887+0800,2014-10-02 00:02:10.887000+08:00,NaN,NaN,每天都有吗？我高三只有周末有时间,0.0,0.0,0.0


In [8]:
user = pd.read_excel('./数据/auth_user.xlsx')

user

,userid,username,date_joined,name,gender,year_of_birth,level_of_education,mailing_address,goals,country,city
0,1,jokey,2013-08-21 13:57:24,shi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,2004011344,2013-08-22 01:57:57,韩文弢,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,WentaoHan,2013-08-22 02:22:07,韩文弢,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,cmouse,2013-08-22 05:48:21,薛宇飞,m,1985.0,p,北京市清华大学,NaN,NaN,NaN
4,6,Yufei_Xue,2013-08-22 06:13:39,薛宇飞,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
256005,326143,chenqian1,2014-10-01 17:44:18,chenqian,f,1993.0,b,NaN,NaN,NaN,NaN
256006,326144,liuxx_908,2014-10-01 17:46:11,刘葵,f,NaN,none,NaN,NaN,NaN,NaN
256007,326145,sjli_152705,2014-10-01 18:05:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256008,326146,Tech-d,2014-10-01 18:07:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
user['level_of_education'].value_counts().index
'female master 1993'

Index(['b', 'm', 'hs', 'a', 'p', 'jhs', 'other', 'none', 'el'], dtype='object', name='level_of_education')

In [7]:
import re
grades = []
for file in os.listdir('./数据/grade/'):
    filepath = f'./数据/grade/{file}'
    df_ = pd.read_excel(filepath)
    usecols = ['userid'] + [x for x in df_.columns if re.findall('Exercise \d+', x)]
    df_ = df_[usecols]
#     ['TsinghuaX/80512073_2014_1X/_2014_', 'TsinghuaX/80512073X/_',
#        'TsinghuaX/80000901_1X/_', 'TsinghuaX/30240184_1X/_',
#        'TsinghuaX/20220332_1X/_', 'TsinghuaX/20220332_2X/_']
    
    if '80512073_2014_1X' in file:
        courseid = 'TsinghuaX/80512073_2014_1X/_2014_'
    elif '80512073X' in file:
        courseid = 'TsinghuaX/80512073X/_'
    elif '80000901_1X' in file:
        courseid = 'TsinghuaX/80000901_1X/_'
    elif '30240184_1X' in file:
        courseid = 'TsinghuaX/30240184_1X/_'
    elif '20220332_1X' in file:
        courseid = 'TsinghuaX/20220332_1X/_'        
    elif '20220332_2X' in file:
        courseid = 'TsinghuaX/20220332_2X/_'

    
    
    df_.insert(1, 'courseid', courseid)
    grades.append(df_)
grade = pd.concat(grades)
grade = grade.T.fillna(method='ffill').T
grade['userid'] = grade['userid'].map(int)


C:\Users\zexuan\AppData\Local\Temp\ipykernel_15332\911146601.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  grade = grade.T.fillna(method='ffill').T


# 相关归一化和编码

In [8]:
# data norm

norm_cols = ['Involvement', 'Concentration',
       'Persistence', 'assignment_time', 'Interaction', 'Positive', 'Negative',
       'Confusion', 'Active', 'Constructive', 'Interactive']
for col in norm_cols:
    normalizer = Normalizer()
    normalizer.fit(data[col])
    data[col] = normalizer.z_score_normalize(data[col])

target_normalizer = Normalizer()
target_normalizer.fit(data['total'])
normalizer_params = {'min_val': target_normalizer.min_val, 'max_val': target_normalizer.max_val,
                     'mean_val': target_normalizer.mean_val, 'std_val': target_normalizer.std_val}
json.dump(normalizer_params, open('./normalizer_params.json', 'w', encoding='utf-8'), ensure_ascii=False,)
data['total'] = target_normalizer.z_score_normalize(data['total'])

# uinfo encoder
for col in ['gender', 'year_of_birth',
       'level_of_education']:
    user[col] = LabelEncoder().fit_transform(user[col])
    
# action_norm
for col in ['video', 'assignment']:
    normalizer = Normalizer()
    normalizer.fit(action[col])
    action[col] = normalizer.z_score_normalize(action[col])

# 表格merge，减少耗时

In [14]:
merge_user = pd.merge(user, data, on=['userid'],how='inner')
merge_user = merge_user.drop_duplicates(subset=['userid'])
merge_forum = pd.merge(forum, data, on=['userid', 'courseid'],how='inner')
action.columns = ['userid', 'courseid', 'date', 'video', 'assignment']
action = action.sort_values(by=['userid', 'courseid', 'date'])
merge_action = pd.merge(action, data, on=['userid', 'courseid'],how='inner')
merge_grade = pd.merge(grade, data, on=['userid', 'courseid'],how='inner')
merge_grade = merge_grade.sort_values(by=['userid', 'courseid'])

In [15]:
merge_grade

,userid,courseid,Exercise 01,Exercise 02,Exercise 03,Exercise 04,Exercise 05,Exercise 06,Exercise 07,Exercise 08,...,assignment_time,Interaction,Positive,Negative,Confusion,Active,Constructive,Interactive,total,Learning_achivement
45220,1,TsinghuaX/20220332_1X/_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.180676,-0.075697,-0.060977,-0.056329,-0.110263,-0.069393,-0.077886,-0.063125,-0.375074,1
58217,1,TsinghuaX/80000901_1X/_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.358114,-0.075697,-0.060977,-0.056329,-0.110263,-0.069393,-0.077886,-0.063125,-0.375074,1
42307,4,TsinghuaX/20220332_1X/_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.301234,-0.075697,-0.060977,-0.056329,-0.110263,-0.069393,-0.077886,-0.063125,-0.311503,1
41645,5,TsinghuaX/20220332_1X/_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.351144,0.151719,0.277330,-0.056329,-0.110263,1.011810,-0.077886,-0.063125,-0.062997,1
46440,5,TsinghuaX/20220332_2X/_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.279054,-0.075697,-0.060977,-0.056329,-0.110263,-0.069393,-0.077886,-0.063125,-0.375074,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19725,211266,TsinghuaX/80512073_2014_1X/_2014_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.358114,-0.075697,-0.060977,-0.056329,-0.110263,-0.069393,-0.077886,-0.063125,-0.375074,1
17227,213432,TsinghuaX/80512073_2014_1X/_2014_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.358114,0.151719,-0.060977,-0.056329,1.041086,-0.069393,-0.077886,0.715235,-0.375074,1
15707,213743,TsinghuaX/80512073_2014_1X/_2014_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.358114,0.606552,-0.060977,-0.056329,3.343783,2.093014,-0.077886,0.715235,-0.375074,1
10145,214659,TsinghuaX/80512073_2014_1X/_2014_,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.358114,0.151719,-0.060977,-0.056329,1.041086,-0.069393,-0.077886,0.715235,-0.375074,1


In [16]:
data.head()

,studentid,userid,courseid,Involvement,Concentration,Persistence,assignment_time,Interaction,Positive,Negative,Confusion,Active,Constructive,Interactive,total,Learning_achivement
0,0,7653,TsinghuaX/20220332_1X/_,2.784847,0.199211,5.187413,4.831140,11.749951,9.749925,11.034746,20.614016,1.011810,13.393174,18.617513,4.774195,0
1,1,4069,TsinghuaX/20220332_1X/_,-0.723452,-0.549809,3.206288,0.631096,1.743633,0.615637,-0.056329,3.343783,4.255422,-0.077886,0.715235,4.436112,0
2,2,19624,TsinghuaX/20220332_1X/_,3.403959,3.601018,4.196851,8.211121,3.108131,0.615637,4.380101,11.403225,-0.069393,3.913539,4.607035,4.389878,0
3,3,7580,TsinghuaX/20220332_1X/_,1.752994,0.288940,4.394963,5.592027,1.743633,1.968865,2.161886,1.041086,-0.069393,2.915683,1.493595,4.332456,0
4,4,8104,TsinghuaX/20220332_1X/_,5.674035,3.204918,8.357215,8.461526,3.108131,1.292251,6.598316,7.949179,1.011810,1.917827,6.942114,4.275072,0


# 转换为字典构建数据集

In [17]:
data_dict = {key:df for key, df in list(data.groupby(['userid', 'courseid'], sort=False))}
forum_dict = {key:df for key, df in list(merge_forum.groupby(['userid', 'courseid']))}
action_dict = {key:df for key, df in list(merge_action.groupby(['userid', 'courseid']))}
user_dict = {key:df for key, df in list(merge_user.groupby('userid'))}
grade_dict = {key:df for key, df in list(merge_grade.groupby(['userid', 'courseid']))}

In [18]:
action_dict[(7653,'TsinghuaX/20220332_1X/_')]

,userid,courseid,date,video,assignment,studentid,Involvement,Concentration,Persistence,assignment_time,Interaction,Positive,Negative,Confusion,Active,Constructive,Interactive,total,Learning_achivement
14101,7653,TsinghuaX/20220332_1X/_,2013-10-19,-0.640488,-0.455447,0,2.784847,0.199211,5.187413,4.83114,11.749951,9.749925,11.034746,20.614016,1.01181,13.393174,18.617513,4.774195,0
14102,7653,TsinghuaX/20220332_1X/_,2013-10-20,0.031801,1.121771,0,2.784847,0.199211,5.187413,4.83114,11.749951,9.749925,11.034746,20.614016,1.01181,13.393174,18.617513,4.774195,0
14103,7653,TsinghuaX/20220332_1X/_,2013-10-28,-0.703074,0.428547,0,2.784847,0.199211,5.187413,4.83114,11.749951,9.749925,11.034746,20.614016,1.01181,13.393174,18.617513,4.774195,0
14104,7653,TsinghuaX/20220332_1X/_,2013-10-29,-0.433905,0.161346,0,2.784847,0.199211,5.187413,4.83114,11.749951,9.749925,11.034746,20.614016,1.01181,13.393174,18.617513,4.774195,0
14105,7653,TsinghuaX/20220332_1X/_,2013-10-30,-0.555970,1.798043,0,2.784847,0.199211,5.187413,4.83114,11.749951,9.749925,11.034746,20.614016,1.01181,13.393174,18.617513,4.774195,0
14106,7653,TsinghuaX/20220332_1X/_,2013-10-30,-0.626478,-0.345209,0,2.784847,0.199211,5.187413,4.83114,11.749951,9.749925,11.034746,20.614016,1.01181,13.393174,18.617513,4.774195,0
14107,7653,TsinghuaX/20220332_1X/_,2013-10-31,-0.546009,3.881761,0,2.784847,0.199211,5.187413,4.83114,11.749951,9.749925,11.034746,20.614016,1.01181,13.393174,18.617513,4.774195,0
14108,7653,TsinghuaX/20220332_1X/_,2013-11-03,1.160164,3.208684,0,2.784847,0.199211,5.187413,4.83114,11.749951,9.749925,11.034746,20.614016,1.01181,13.393174,18.617513,4.774195,0
14109,7653,TsinghuaX/20220332_1X/_,2013-11-04,-0.703074,-0.306638,0,2.784847,0.199211,5.187413,4.83114,11.749951,9.749925,11.034746,20.614016,1.01181,13.393174,18.617513,4.774195,0
14110,7653,TsinghuaX/20220332_1X/_,2013-11-06,-0.703074,-0.391830,0,2.784847,0.199211,5.187413,4.83114,11.749951,9.749925,11.034746,20.614016,1.01181,13.393174,18.617513,4.774195,0


In [19]:
grade_dict[(1,
  'TsinghuaX/20220332_1X/_')].iloc[:,2:43]

,Exercise 01,Exercise 02,Exercise 03,Exercise 04,Exercise 05,Exercise 06,Exercise 07,Exercise 08,Exercise 09,Exercise 10,...,Exercise 32,Exercise 33,Exercise 34,Exercise 35,Exercise 36,Exercise 37,Exercise 38,Exercise 39,Exercise 40,Exercise 41
45220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
dataset = []
cid = 0
for key, df in tqdm(data_dict.items()):
    curr_dataset = {}
    curr_dataset['cid'] = str(cid)
    curr_dataset['user-course'] = str(key[0]) + '-' + key[1]
    curr_dataset['score'] = df['total'].values[0]
    curr_dataset['label'] = 1 - float(df['Learning_achivement'].values[0])
    curr_dataset['full'] = df[['Involvement', 'Concentration',
       'Persistence', 'assignment_time', 'Interaction', 'Positive', 'Negative',
       'Confusion', 'Active', 'Constructive', 'Interactive',]].values.squeeze().tolist()
    curr_action = action_dict.get(key)
    curr_dataset['action'] = curr_action [['video', 'assignment']].values.tolist() if curr_action is not None else [[0.,0.]]
    curr_dataset['uinfo'] = user_dict.get(key[0], [])[['gender', 'year_of_birth',
       'level_of_education']].values.squeeze().tolist()
    curr_dataset['grade'] = grade_dict.get(key).iloc[:,2:43].values.tolist()
    
    curr_forum = forum_dict.get(key)
    forum_text = curr_forum['body'].tolist() if curr_forum is not None else ['']
    forum_text = [str(x) for x in forum_text]
    curr_dataset['forum_text'] = forum_text
    curr_dataset['forum_vec'] = bert_representation(forum_text)

     

    
    dataset.append(curr_dataset)
    cid += 1
#     break

100%|████████████████████████████████████████████████████████████████████████████| 59581/59581 [10:28<00:00, 94.79it/s]


In [21]:
write_jsonl(dataset, './data/corpus.json')

In [ ]:
a = [1,2,3,4,5,]
i = 4
a[:4]+a[4+1:]